In [65]:
## Importing Libraries ##
import pandas as pd
import numpy as np
import seaborn as sns
import xlrd
import openpyxl
from datetime import datetime
from sklearn import *
import copy

In [66]:
## Variable declaration ##
df, cf, rf = pd.read_csv('eventDatasets'), pd.read_csv('AccumusDatasets'), pd.read_csv('riverHeightDatasets')

RGEO, CGEO = rf[['LAT','LONG']].value_counts().index, cf[['LAT','LONG']].value_counts().index

In [67]:
## Converting minutes and seconds to hours ##
rf['OBS_TIME_LOC'] = pd.to_datetime(rf['OBS_TIME_LOC'])
rf['OBS_TIME_LOC'] = pd.to_datetime(rf['OBS_TIME_LOC'].dt.strftime('%Y-%m-%d %H'))

rf['PRCP_HOUR'] = rf.groupby([rf['LOC_NAME'].values,rf['OBS_TIME_LOC'].dt.date])['PRCP'].transform('max')
rf = rf.drop(['VOID_FLAG','Unnamed: 0','PRCP'],axis=1)

cf['OBS_TIME_LOC'] = pd.to_datetime(cf['OBS_TIME_LOC'])
cf['OBS_TIME_LOC'] = pd.to_datetime(cf['OBS_TIME_LOC'].dt.strftime('%Y-%m-%d %H'))

cf['HT_MEAN'] = cf.groupby([cf['LOC_NAME'].values,cf['OBS_TIME_LOC'].dt.date,cf['OBS_TIME_LOC'].dt.hour])['HT'].transform('max')
cf = cf.drop(['VOID_FLAG','Unnamed: 0','HT'],axis=1 )

cf, rf = cf.drop_duplicates(), rf.drop_duplicates()

In [68]:
## Creating a new dataframe that can be merged ##
newList = copy.copy(cf[['HT_MEAN','OBS_TIME_LOC','LAT','LONG']])

for j in range(0,len(CGEO)):
        dst, smallDist, values = 9999,9999999999, []
        
        for i in range(0,len(RGEO)):
            dst = distance.euclidean(CGEO[j],RGEO[i]) 
            
            if dst < smallDist: 
                smallDist, values = dst, RGEO[i]
            
        newList['LAT'] = newList['LAT'].replace({CGEO[j][0]:values[0]})
        newList['LONG'] = newList['LONG'].replace({CGEO[j][1]:values[1]})

In [91]:
## Merging values ##
newList, rf = newList.sort_values(by='OBS_TIME_LOC'), rf.sort_values(by='OBS_TIME_LOC')

merged_rf_cf = pd.merge_asof(rf,newList, on = 'OBS_TIME_LOC',by=['LAT','LONG'])
merged_rf_cf = merged_rf_cf.sort_values(by=['LAT','LONG','OBS_TIME_LOC'])
merged_rf_cf= merged_rf_cf.bfill().ffill()
newList.HT_MEAN.value_counts()

 0.350     7830
 0.360     7761
 0.850     7539
 2.710     7422
 0.860     7291
           ... 
 17.040       1
 13.920       1
-0.739        1
 10.140       1
 2.129        1
Name: HT_MEAN, Length: 5031, dtype: int64

In [104]:
merged_rf_cf.to_csv('merged_rf_cf.csv')